### **Notebook 1: Setting Up S3 Mounts and Delta Lake Bronze Layer**

#### Steps:
1. Mount the S3 bucket.
2. Load raw data from S3 and save it as a Delta table in the Bronze layer.
3. Register the Bronze table in Unity Catalog.

#### Code:

In [0]:
# Step 1: Mount S3 bucket
dbutils.fs.mount(
    source="s3a://your-bucket-name/retail-data/by-day/",
    mount_point="/mnt/s3data/retail-data/by-day",
    extra_configs={"fs.s3a.awsAccessKeyId": "<YOUR_ACCESS_KEY>",
                   "fs.s3a.awsSecretAccessKey": "<YOUR_SECRET_KEY>"}
)

# Step 2: Verify the mount
display(dbutils.fs.ls("/mnt/s3data/retail-data/by-day/"))

# Step 3: Load raw data and save as Bronze Delta table
raw_data = spark.read.format("csv").option("header", "true").load("/mnt/s3data/retail-data/by-day/")
bronze_path = "/mnt/s3data/bronze/retail_data"
raw_data.write.format("delta").mode("overwrite").save(bronze_path)

# Step 4: Register the Bronze table in Unity Catalog
spark.sql(f"""
CREATE TABLE IF NOT EXISTS retail_catalog.bronze.retail_data
USING DELTA
LOCATION '{bronze_path}'
""")

---